This notebook  illustrates how you can use the langchain with custom pdf data & chat it using autogen agent

In [1]:
%pip install pyautogen~=0.1.0  langchain langchain_community openai tiktoken lancedb pypdf -q -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [2]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain_community.vectorstores import LanceDB

Requirements
AutoGen requires Python>=3.8. To run this notebook example, please install pyautogen:


In [3]:
!pip install pyautogen

In [38]:
import lancedb
import os

# setup OPENAI API KEY
os.environ["OPENAI_API_KEY"] = "sk-...."

embeddings = OpenAIEmbeddings()

In [5]:
# !wget -O uniswap_v3.pdf https://uniswap.org/whitepaper-v3.pdf
!wget -O food.pdf https://pdf.usaid.gov/pdf_docs/PA00TBCT.pdf

--2024-05-28 05:17:50--  https://pdf.usaid.gov/pdf_docs/PA00TBCT.pdf
Resolving pdf.usaid.gov (pdf.usaid.gov)... 23.4.180.157, 2600:1408:5400:197::1923, 2600:1408:5400:183::1923
Connecting to pdf.usaid.gov (pdf.usaid.gov)|23.4.180.157|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6419525 (6.1M) [application/pdf]
Saving to: ‘food.pdf’

food.pdf            100%[===================>]   6.12M  27.0MB/s    in 0.2s    

2024-05-28 05:17:51 (27.0 MB/s) - ‘food.pdf’ saved [6419525/6419525]



# create file name with OAI_CONFIG_LIT.json

In [53]:
# create file name with OAI_CONFIG_LIT.
import json

config = [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]

with open("OAI_CONFIG_LIT.json", "w") as fp:
    json.dump(config, fp)

**create OAI_CONFIG_LIST.json file in pwd & upload
in it**


[
   {
     "model": "gpt-4",
     "api_key": "sk-yourapikey"
   }
]

In [41]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST.json",
    filter_dict={
        "model": ["gpt-4"],
    },
)

In [42]:
loaders = [PyPDFLoader("./food.pdf")]
docs = []
for l in loaders:
    docs.extend(l.load())
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(docs)

In [43]:
import lancedb

embeddings = OpenAIEmbeddings()

vectorstore = LanceDB.from_documents(documents=docs, embedding=embeddings)

In [45]:
qa = ConversationalRetrievalChain.from_llm(
    OpenAI(
        temperature=0,
    ),
    vectorstore.as_retriever(),
    memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True),
)

In [46]:
def answer_food_question(question):
    response = qa({"question": question})
    return response["answer"]

In [47]:
question = "what is good food"
answer_food_question(question)

' Good food is any type of food that provides the recommended amounts of nutrients for the body to perform its physiological activities. It should be eaten at the right time, in the right amounts, and prepared correctly. Good food is important for physical and cognitive development, and can help prevent health problems. Foods can also be classified according to their functions in the body, such as energy-giving foods, body-building foods, and protective foods.'

In [18]:
!pip install pyautogen

#### 4. Set up AutoGen user agent and assistant agent with function calling enabled.

In [48]:
llm_config = {
    "request_timeout": 600,
    "seed": 42,
    "config_list": config_list,
    "temperature": 0,
    "functions": [
        {
            "name": "answer_food_question",
            "description": "Answer to any quetion related to food & provided answer_food_question ",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "The question to ask in relation to food",
                    }
                },
                "required": ["question"],
            },
        }
    ],
}

In [49]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": "."},
    llm_config=llm_config,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""",
    function_map={"answer_food_question": answer_food_question},
)

In [50]:
# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""
what is good food?
""",
)

user_proxy (to assistant):


what is good food?


--------------------------------------------------------------------------------
assistant (to user_proxy):

***** Suggested function Call: answer_food_question *****
Arguments: 
{
  "question": "what is good food?"
}
*********************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION answer_food_question...
user_proxy (to assistant):

***** Response from calling function "answer_food_question" *****
 Good food is food that is able to provide the recommended amounts of nutrients for the body to perform all its physiological activities. It is important because it helps with physical and cognitive development, promotes good health, and improves the quality of life. Good food should be eaten at the right time, in the right amounts, and prepared correctly. It can also be classified into different categories based on its function in the body,

In [51]:
# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""
please explain me essential minerals, sources, functions and symptoms of
deficiency?
""",
)

user_proxy (to assistant):


please explain me essential minerals, sources, functions and symptoms of
deficiency?


--------------------------------------------------------------------------------
assistant (to user_proxy):

Sure, here are some essential minerals, their sources, functions, and symptoms of deficiency:

1. Calcium:
   - Sources: Dairy products, leafy green vegetables, fish with edible bones (like sardines and salmon), fortified foods.
   - Functions: Necessary for bone health, muscle function, nerve transmission, blood clotting.
   - Deficiency Symptoms: Osteoporosis, rickets in children, muscle cramps, dental problems.

2. Iron:
   - Sources: Red meat, poultry, fish, legumes, fortified cereals.
   - Functions: Essential for the production of red blood cells, carries oxygen in the blood.
   - Deficiency Symptoms: Anemia, fatigue, weakness, pale skin, shortness of breath.

3. Potassium:
   - Sources: Bananas, oranges, cantaloupes, raisins, nuts, fish, chicken, beef, and p

In [52]:
# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""
which food Keeps eyes healthy ?
""",
)

user_proxy (to assistant):


which food Keeps eyes healthy ?


--------------------------------------------------------------------------------
assistant (to user_proxy):

***** Suggested function Call: answer_food_question *****
Arguments: 

{
  "question": "which food Keeps eyes healthy ?"
}
*********************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION answer_food_question...
user_proxy (to assistant):

***** Response from calling function "answer_food_question" *****
 Fruits and vegetables, particularly dark green leafy vegetables and yellow fruits, are considered protective and can help keep eyes healthy.
*****************************************************************

--------------------------------------------------------------------------------
assistant (to user_proxy):

Fruits and vegetables, particularly dark green leafy vegetables and yellow fruits, are considered pr